In [1]:
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude,longitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1758078688,2025-09-17 00:11,1758078000,2025-09-17 00:00,16.4,61.5,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003,4.0,6.5,94,E,1020.0,30.12,0.0,0.0,100,75,16.4,61.5,13.2,55.7,13.3,55.9,12.6,54.7,8.0,4.0,0.0,6.8,11.0,"[{'date': '2025-09-17', 'date_epoch': 17580672..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-17,1758067200,"[{'time_epoch': 1758078000, 'time': '2025-09-1...",25.8,78.4,13.2,55.7,17.4,63.3,9.4,15.1,0.0,0.0,0.0,9.0,5.0,72,0,0,0,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,2.2,06:12 AM,06:11 PM,03:36 AM,02:24 PM,Waning Crescent,23,0,0


In [5]:
hour_list = dados["forecast"]["forecastday"][0]["hour"]
df_hour = pd.json_normalize(hour_list)
df_hour


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.text,condition.icon,condition.code
0,1758078000,2025-09-17 00:00,13.6,56.4,0,3.8,6.1,87,E,1017.0,30.03,0.0,0.0,0.0,97,93,13.5,56.4,13.5,56.4,13.6,56.4,13.1,55.5,0,0,0,0,2.0,1.0,6.6,10.7,0.0,Neblina,//cdn.weatherapi.com/weather/64x64/night/143.png,1030
1,1758081600,2025-09-17 01:00,16.4,61.5,0,4.0,6.5,94,E,1020.0,30.12,0.0,0.0,0.0,100,75,13.2,55.7,13.2,55.7,13.3,55.9,12.6,54.7,0,0,0,0,8.0,4.0,6.8,11.0,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
2,1758085200,2025-09-17 02:00,13.2,55.7,0,4.3,6.8,98,E,1016.0,30.01,0.0,0.0,0.0,94,85,13.0,55.3,13.0,55.3,13.2,55.7,12.3,54.1,0,0,0,0,2.0,1.0,6.9,11.1,0.0,Neblina,//cdn.weatherapi.com/weather/64x64/night/143.png,1030
3,1758088800,2025-09-17 03:00,13.2,55.8,0,4.5,7.2,106,ESE,1017.0,30.02,0.0,0.0,0.0,92,75,12.9,55.2,12.9,55.2,13.2,55.8,12.0,53.6,0,0,0,0,10.0,6.0,6.9,11.2,0.0,Nublado,//cdn.weatherapi.com/weather/64x64/night/119.png,1006
4,1758092400,2025-09-17 04:00,13.3,55.9,0,4.7,7.6,105,ESE,1017.0,30.03,0.0,0.0,0.0,91,71,12.9,55.2,12.9,55.2,13.3,55.9,11.9,53.4,0,0,0,0,10.0,6.0,7.0,11.2,0.0,Nublado,//cdn.weatherapi.com/weather/64x64/night/119.png,1006
5,1758096000,2025-09-17 05:00,13.4,56.1,0,4.7,7.6,102,ESE,1018.0,30.05,0.0,0.0,0.0,90,61,13.1,55.5,13.1,55.5,13.4,56.1,11.8,53.2,0,0,0,0,10.0,6.0,6.6,10.6,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
6,1758099600,2025-09-17 06:00,13.5,56.4,0,4.5,7.2,93,E,1018.0,30.07,0.0,0.0,0.0,88,54,13.3,55.9,13.3,55.9,13.5,56.4,11.6,52.9,0,0,0,0,10.0,6.0,6.0,9.7,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
7,1758103200,2025-09-17 07:00,14.5,58.1,1,4.3,6.8,93,E,1019.0,30.09,0.0,0.0,0.0,84,41,14.5,58.0,14.5,58.0,14.5,58.1,11.9,53.3,0,0,0,0,10.0,6.0,5.1,8.2,0.1,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/day/116.png,1003
8,1758106800,2025-09-17 08:00,16.2,61.1,1,4.3,6.8,91,E,1019.0,30.09,0.0,0.0,0.0,76,13,16.2,61.1,16.2,61.1,16.2,61.1,12.1,53.7,0,0,0,0,10.0,6.0,4.9,7.9,0.8,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
9,1758110400,2025-09-17 09:00,18.9,66.1,1,4.7,7.6,89,E,1019.0,30.09,0.0,0.0,0.0,63,16,18.9,66.1,18.9,66.1,18.9,66.1,11.8,53.2,0,0,0,0,10.0,6.0,5.4,8.7,2.5,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000


In [6]:
# Lista apenas com as colunas importantes
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
    "condition.text"
]

df_limpo = df_hour[colunas_importantes]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain,condition.text
16,2025-09-17 16:00,21.3,57,5,2.4,15.1,0,0,Sol
9,2025-09-17 09:00,18.9,63,16,2.5,7.6,0,0,Sol
5,2025-09-17 05:00,13.1,90,61,0.0,7.6,0,0,Parcialmente nublado
22,2025-09-17 22:00,13.6,83,24,0.0,12.2,0,0,Céu limpo
10,2025-09-17 10:00,21.6,50,3,5.8,7.2,0,0,Sol


In [7]:
# Cria uma tabela associando estilos de roupa com dados do clima
df_tabela1 = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
    "condição do dia": df_limpo ["condition.text"].values[:4]
})

df_tabela1

,estilos,data e hora,temperatura,chance de chuva,umidade,uv,condição do dia
0,casual,2025-09-17 00:00,13.5,0,97,0.0,Neblina
1,esportivo,2025-09-17 01:00,13.2,0,100,0.0,Parcialmente nublado
2,social,2025-09-17 02:00,13.0,0,94,0.0,Neblina
3,streetwear,2025-09-17 03:00,12.9,0,92,0.0,Nublado


In [8]:
#subdados para peças de roupas por estilo especifico

# subdados_roupas = {
#     "casual": {
#         "casual para calor": ["camisetas polo", "camisetas básicas de algodão", "camisas de manga curta"],
#         "casual para frio": ["suéter leve", "jaqueta jeans", "jaqueta de sarja"],
#         "casual meia estação": ["camisa xadrez", "camisa de botões leve", "cardigan fino"],
#     },
#     "esportivo": {
#         "esportivo para calor": ["camisetas dry-fit", "regatas, shorts esportivos", "bermudas de tecido leve"],
#         "esportivo para frio": ["moletom com capuz", "calça jogger", "calça de moletom"],
#         "esportivo meia estação": ["blusa térmica fina", "calça legging", "jaqueta leve de treino"],
#     },
#     "social": {
#         "social para calor": ["camisas de linho", "camisas de manga curta sociais", "calças de alfaiataria leve"],
#         "social para frio": ["blazer estruturado", "suéter de lã", "colete de alfaiataria"],
#         "social meia estação": ["trench coat leve", "camisa social slim fit", "calça social reta"],
#     },
#     "streetwear": {
#         "streetwear para calor": ["camisetas oversized", "camisas estampadas leves", "bermudas cargo"],
#         "streetwear para frio": ["hoodies com estampas", "jaquetas bomber", "jaquetas jeans oversized"],
#         "streetwear meia estação": ["jaqueta utilitária", "camisa xadrez oversized", "calça jogger estilizada"],
#     }
# }

In [9]:
print(df_tabela1.columns.tolist())

['estilos', 'data e hora', 'temperatura', 'chance de chuva', 'umidade', 'uv', 'condição do dia']


In [10]:
#transformando variáveis e em um número categorico 

x = pd.get_dummies(df_tabela1.drop(columns=["estilos"]))
y = df_tabela1["estilos"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [13]:
# exemplo: 22°C, 70% umidade, UV 5, vento 10, sem chuva
novo_exemplo = pd.DataFrame([{
    "temperatura": 22,
    "chance_de_chuva": 0,
    "umidade": 70,
    "uv": 5
}])


In [14]:

# print("Recomendação:", clf.score(X_test, y_test))